In [1]:
import pymongo as pm
import json
import pandas as pd

## Stimuli Upload
`ratchet_id` refers to a set of rules to be queried for trials. Each collection of `ratchet_id` has a `_dev` version, except for `dev` collection itself.

Current Values:
1. `dev`: 2 single features rules, 2 conjunctive rules, 2 disjunctive rules, 2 conjunctive disjunction rules, & 2 disjunctive conjunction rules
2. `50_rules`: 10 single feature rules, 10 conjunctive rules, 10 disjunctive rules, 10 conjunctive disjunction rules, & 10 disjunctive conjunction rules

In [2]:
def gen_vars(local_machine=True):
    if local_machine:
        pswd = ""
        user = ""
        host = "localhost"
    else:
        auth = pd.read_csv('auth.csv', header = None)
        pswd = auth.values[0][0].strip()
        user = 'sketchloop'
        host = 'localhost' 
    return user, pswd, host

In [3]:
def get_creature_names(rule_type):
    d = {
        "SINGLE_FEATURE": ["morseth", "morseths"],
        "CONJUNCTION": ["oller", "ollers"],
        "DISJUNCTION": ["kwep", "kweps"],
        "CONJUNCTION_DISJUNCTION": ["zorb", "zorbs"],
        "DISJUNCTION_CONJUNCTION":["luzak", "luzaks"],        
    }
    return d[rule_type]

In [4]:
def convert_concept_summary_into_db_data(data):
    db_data = {
        "SINGLE_FEATURE": [],
        "CONJUNCTION": [],
        "DISJUNCTION": [],
        "CONJUNCTION_DISJUNCTION": [],
        "DISJUNCTION_CONJUNCTION": [],
    }
    for rule_idx, rule in data.items():
        rule_type = rule["type"]
        speciesName = get_creature_names(rule_type)[0]
        speciesNamePlural = get_creature_names(rule_type)[1]
        db_data[rule_type].append({
            "rule_idx": int(rule_idx),
            "file_name": rule["name"],
            "name": rule["phrase"],
            "games": [],
            "numGames": 0,
            "speciesName": speciesName,
            "speciesNamePlural": speciesNamePlural,
        })
    return db_data

In [5]:
def load_data_into_db(concept_summary_file, col_prefix):
    with open(concept_summary_file) as f:
        data = json.load(f)
        db_data = convert_concept_summary_into_db_data(data)
        for rule_type, rules in db_data.items():
            col = col_prefix.format(rule_type)
            for rule in rules:
                db[col].insert_one(rule)

In [6]:
# Set Variables 
user, pswd, host = gen_vars(local_machine=False)

if user == '' or pswd == '':
    con_info = 'mongodb://{}'.format(host)   
else:
    con_info = 'mongodb://{}:{}@{}:27017'.format(user, pswd, host)

conn = pm.MongoClient(con_info)
db = conn['genGames']

In [7]:
# Print Existing DB
print(sorted(db.list_collection_names()))

['fifty_rules_CONJUNCTION', 'fifty_rules_CONJUNCTION_DISJUNCTION', 'fifty_rules_DISJUNCTION', 'fifty_rules_DISJUNCTION_CONJUNCTION', 'fifty_rules_SINGLE_FEATURE', 'mpGame3', 'mpGame4', 'pilot_fifty_rules_CONJUNCTION', 'pilot_fifty_rules_CONJUNCTION_DISJUNCTION', 'pilot_fifty_rules_DISJUNCTION', 'pilot_fifty_rules_DISJUNCTION_CONJUNCTION', 'pilot_fifty_rules_SINGLE_FEATURE']


In [8]:
# Drop Existing Collections
x = sorted(db.list_collection_names())
exclude = ['mpGame3', 'mpGame4', 'system.indexes']
for i in x:
    if i not in exclude:
        db[i].drop()

In [9]:
# Print Existing DB
print(sorted(db.list_collection_names()))

['mpGame3', 'mpGame4']


In [10]:
# Load 50 Rules Data (Pilot)
fifty_rules_col_prefix = 'pilot_fifty_rules_{}'
fifty_rules_concept_summary_file = './fifty_rules/concept_summary.json'
load_data_into_db(fifty_rules_concept_summary_file, fifty_rules_col_prefix)

In [11]:
# Load 50 Rules Data
fifty_rules_col_prefix = 'fifty_rules_{}'
fifty_rules_concept_summary_file = './fifty_rules/concept_summary.json'
load_data_into_db(fifty_rules_concept_summary_file, fifty_rules_col_prefix)

In [12]:
sorted(db.list_collection_names())

['fifty_rules_CONJUNCTION',
 'fifty_rules_CONJUNCTION_DISJUNCTION',
 'fifty_rules_DISJUNCTION',
 'fifty_rules_DISJUNCTION_CONJUNCTION',
 'fifty_rules_SINGLE_FEATURE',
 'mpGame3',
 'mpGame4',
 'pilot_fifty_rules_CONJUNCTION',
 'pilot_fifty_rules_CONJUNCTION_DISJUNCTION',
 'pilot_fifty_rules_DISJUNCTION',
 'pilot_fifty_rules_DISJUNCTION_CONJUNCTION',
 'pilot_fifty_rules_SINGLE_FEATURE']